<a href="https://colab.research.google.com/github/FridBlake/Machine-Learning-Showcase/blob/main/The_Titanic_Dataset_from_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#these two modules will help us to graph the data
import matplotlib.pyplot as plt
import seaborn as sns

#pandas will help us visualise the data and manipulate it so that we can clean and treat it.
import pandas as pd
import numpy as np
#put this on whenver you use matplotlib
%matplotlib inline


In [ ]:
#we are strong a PANDAS object of type 'DataFrame' in a variable data, which will use a .csv file.
data = pd.read_csv('train (1).csv')
data

Imputation


In [ ]:
#process by which missing values are replaced with the most likely ones.

#.isnull() shows you a DataFrame, with T/F to show if the data is missing. .sum() shows you the sum for all missing data in a column.
data.isnull().sum()

#when you want to isolate a single column, use data[['Column name here']]. .median() finds the median of data the column.
medianage = data['Age'].median()

#.fillna() fills the missing data, taking in the 1st parameter as what you want to replace the data with, and inplace=True, if you want the original data variable to be modified.
data['Age'].fillna(medianage, inplace=True)

data

In [ ]:
for i, j in enumerate(['SibSp', 'Parch', 'Age']):
  plt.figure(i)
  sns.catplot(x=j, y='Survived', data= data, kind='point', aspect=2)


for i, j in enumerate(['Sex']):
  plt.figure(i)
  sns.catplot(x=j, y='Survived', data= data, kind='box', aspect=2)

In [ ]:
'''since we now see that there is a relationship (inverse) beteween the number of family members they have in total and the survival rate,
we can go ahead and add them up into one column, so as to prevent collinearity. - Plus, we also know that age is not really following a pattern at all.'''

data['fam_ct'] = data['SibSp'] + data['Parch']

#now we can just go ahead and get rid of SibSp and PARCH.

data.drop('SibSp', axis=1, inplace=True)
data.drop('Parch', axis=1, inplace=True)
data.drop('PassengerId', axis=1, inplace=True)

data


Now, I would like to go ahead and work with the categorical variables, and change them.

In [ ]:
# I replace sex with binary numbers.
data['Sex'] = data['Sex'].map({'male' : 0, 'female' : 1})

#Now, I would like to see if cabin is something that has been randomly missing.

data.groupby(data['Cabin'].isnull())['Survived'].mean()

#When I ran that the data for cabin is not missing in random. I continued on without showing the output as I edited the code block already. Those without a cabin were much more likely to survive.
#In such a case, np.where() could be useful.

#if they don't have a cabin, it's a 0, else it's a 1.
data[['hasCabin']] = np.where(data[['Cabin']].isnull(), 0, 1)
data.drop('Cabin', axis=1, inplace=True)

#now I would like to do the same thing for data['Embarked'] and see if it makes a difference at all.

data.groupby(data['Embarked'])['Survived'].mean()

#I don't think where they embarked WOULD have a big effect, if any it'll be NEGLIGIBLE.

data



In [ ]:
#So I drop the ones that we do not need.
data.drop('Embarked', axis=1, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.drop('Ticket', axis=1, inplace=True)

In [ ]:

#this is what we are left with now.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = data.drop('Survived', axis=1)
y = data[['Survived']]

cNames = X.columns

#data has now been normalised
X = pd.DataFrame(MinMaxScaler().fit_transform(X), columns=cNames)
X
y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
#now 30 percent of the data is in testing group.

In [ ]:

data.groupby(['fam_ct'])['Survived'].median()

In [ ]:
X

In [ ]:
#Let us first try out a few different classification methods, and then go with the best.
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
RC_model = RandomForestClassifier()

parameters = {'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'max_depth' : [1, 2, 3, 4, 5, 6]}

cv = GridSearchCV(RC_model, parameters, cv=5)
cv.fit(X_train, y_train)


In [ ]:
cv.best_estimator_
#with this we can find that this random forest classifier works best at a max depth of 6 and n_est = 20.

In [ ]:
#The random forst model has an accuracy of 87 percent.

rf_Model = RandomForestClassifier(max_depth = 6, n_estimators = 20)
rf_Model.fit(X_train, y_train)

y_pred = rf_Model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

In [ ]:

#Now, let us try a multilayer perceptron.

import tensorflow as tf
from tensorflow import keras

nn_Model = keras.Sequential([
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

cb = keras.callbacks.EarlyStopping(min_delta = 0.01, patience=3)

nn_Model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = nn_Model.fit(X_train, y_train, epochs=100, callbacks = cb, validation_data=(X_test, y_test), batch_size=32)

plt.plot(history.history['loss'], color='blue')
plt.plot(history.history['val_loss'], color='yellow')
plt.show()

y_pred = nn_Model.predict(X_test)


In [ ]:
'''
now that is only as far as i can get it to go. Neural networks are rather notorious to train indeed.
I want to try naive_bayes, GaussianNB to be specific. - Again, Gaussian NB turns out to be terrible in this case.
'''

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_test, y_test)

y_pred = nb_model.predict(X_test)

print(classification_report(y_pred, y_test))


Now let's try out logistic regression - used for data that's rather 'behaved' - in other words, it's not made of any outliers.


In [ ]:
from sklearn.linear_model import LogisticRegression
lr_Model = LogisticRegression(random_state=1)
lr_Model.fit(X_train, y_train)
y_pred = lr_Model.predict(X_test)

print(classification_report(y_pred, y_test))

# Okee-doke, we seem to have hit an end when it comes to Logistic Regression. it gives us almost the same model as we got with the Neural network.
#Let's note that logistic regression is deemed as one of the best models for the Titanic dataset. I wouldn't bother with SVM because the data isn't 'short and fat'



In [ ]:
#the LAST model I want to bother with is kNN.

from sklearn.neighbors import KNeighborsClassifier

knn_Model = KNeighborsClassifier(7)
knn_Model.fit(X_train, y_train)

y_pred2 = knn_Model.predict(X_test)

print(classification_report(y_pred2, y_test))


**Concluding remarks:**

While we saw that the accuracy hit an 77 per cent in the Logistic Regression model, that is the accuracy I would like to go with, for now. However, I have **just** about made it. - This varied with multiple attempts. The highest I got in a logistic regression for this dataset was an 81 percent, 82 for KNN with 5 neighbours.

It is important to know that the dataset is imbalanced, with more people in class 'not survived', and less in 'survived'. Moreover, if I had more time, I would have used VIF (Variance inflation factor) to check if I could find more multicollinearity.

-Advaith Subramanian Sahasranamam